## 1. 获取服务
- 在 Native 层的服务注册，我们选择以 media 为例来展开讲解，先来看看 media 的类关系图

### 1.1 类图
![image](binder6_page1.png)

#### 图解
- 蓝色: 代表获取 MediaPlayerService 服务相关的类；
- 绿色: 代表 Binder 架构中与 Binder 驱动通信过程中的最为核心的两个类；
- 紫色: 代表注册服务和获取服务的公共接口/父类；

---

## 2. 获取 Media 服务
### 2.1 getMediaPlayerService
- 返回 BpServiceManager
- 在请求获取名为 "media.player" 的服务过程中，采用不断循环获取的方法。由于 MediaPlayerService 服务可能还没向 ServiceManager 注册完成或者尚未启动完成等情况，故则 binder 返回为 NULL，休眠 0.5s 后继续请求，直到获取服务为止

```
sp<IMediaPlayerService>&IMediaDeathNotifier::getMediaPlayerService()
{
    Mutex::Autolock _l(sServiceLock);
    if (sMediaPlayerService == 0) {
        sp<IServiceManager> sm = defaultServiceManager(); // 获取 ServiceManager
        sp<IBinder> binder;
        do {
            // 获取名为"media.player"的服务 【见2.2】
            binder = sm->getService(String16("media.player"));
            if (binder != 0) {
                break;
            }
            usleep(500000); // 0.5s
        } while (true);

        if (sDeathNotifier == NULL) {
            sDeathNotifier = new DeathNotifier(); // 创建死亡通知对象
        }

        // 将死亡通知连接到 binder 【见流程14】
        binder->linkToDeath(sDeathNotifier);
        sMediaPlayerService = interface_cast<IMediaPlayerService>(binder);
    }
    return sMediaPlayerService;
}
```

---

### 2.2 BpSM.getService
- 通过 BpServiceManager 来获取 MediaPlayer 服务：检索服务是否存在，当服务存在则返回相应的服务，当服务不存在则休眠1s再继续检索服务。
- 该循环进行 5 次。为什么是循环 5 次呢，这估计跟 Android 的 ANR 时间为 5s 相关。如果每次都无法获取服务，循环 5 次，每次循环休眠 1s，忽略 checkService() 的时间，差不多就是 5s 的时间

```
virtual sp<IBinder> getService(const String16& name) const
{
    unsigned n;
    for (n = 0; n < 5; n++){
        sp<IBinder> svc = checkService(name); //【见2.3】
        if (svc != NULL) return svc;
        sleep(1);
    }
    return NULL;
}
```

---

### 2.3 BpSM.checkService
- 检索指定服务是否存在, 其中 remote() 为 BpBinder

```
virtual sp<IBinder> checkService( const String16& name) const
{
    Parcel data, reply;
    // 写入 RPC 头
    data.writeInterfaceToken(IServiceManager::getInterfaceDescriptor());
    // 写入服务名
    data.writeString16(name);
    remote()->transact(CHECK_SERVICE_TRANSACTION, data, &reply); //【见2.4】
    return reply.readStrongBinder(); //【见小节2.9】
}
```

---

### 2.4 BpBinder::transact
- Binder 代理类调用 transact() 方法，真正工作还是交给 IPCThreadState 来进行 transact 工作

```
status_t BpBinder::transact(
    uint32_t code, const Parcel& data, Parcel* reply, uint32_t flags)
{
    if (mAlive) {
        //【见流程2.5】
        status_t status = IPCThreadState::self()->transact(
            mHandle, code, data, reply, flags);
        if (status == DEAD_OBJECT) mAlive = 0;
        return status;
    }
    return DEAD_OBJECT;
}
```

#### 2.4.1 IPCThreadState::self
- TLS 是指 Thread local storage(线程本地储存空间)，每个线程都拥有自己的 TLS，并且是私有空间，线程之间不会共享
- 通过 pthread_getspecific/pthread_setspecific 函数可以获取/设置这些空间中的内容
- 从线程本地存储空间中获得保存在其中的 IPCThreadState 对象

```
IPCThreadState* IPCThreadState::self()
{
    if (gHaveTLS) {
restart:
        const pthread_key_t k = gTLS;
        IPCThreadState* st = (IPCThreadState*)pthread_getspecific(k);
        if (st) return st;
        return new IPCThreadState;  // 初始 IPCThreadState 【见小节2.4.2】
    }

    if (gShutdown) return NULL;
    pthread_mutex_lock(&gTLSMutex);
    if (!gHaveTLS) {
        // 首次进入 gHaveTLS 为 false
        if (pthread_key_create(&gTLS, threadDestructor) != 0) {
            // 创建线程的 TLS
            pthread_mutex_unlock(&gTLSMutex);
            return NULL;
        }
        gHaveTLS = true;
    }
    pthread_mutex_unlock(&gTLSMutex);
    goto restart;
}
```

#### 2.4.2 IPCThreadState初始化
- 每个线程都有一个 IPCThreadState，每个 IPCThreadState 中都有一个 mIn、一个 mOut。成员变量 mProcess 保存了 ProcessState 变量(每个进程只有一个)
  - mIn 用来接收来自 Binder 设备的数据，默认大小为 256 字节
  - mOut 用来存储发往 Binder 设备的数据，默认大小为 256 字节

```
IPCThreadState::IPCThreadState()
    : mProcess(ProcessState::self()),
      mMyThreadId(gettid()),
      mStrictModePolicy(0),
      mLastTransactionBinderFlags(0)
{
    pthread_setspecific(gTLS, this);
    clearCaller();
    mIn.setDataCapacity(256);
    mOut.setDataCapacity(256);
}
```

---

### 2.5 IPC::transact
```
status_t IPCThreadState::transact(int32_t handle,
                                  uint32_t code, const Parcel& data,
                                  Parcel* reply, uint32_t flags)
{
    status_t err = data.errorCheck(); // 数据错误检查
    flags |= TF_ACCEPT_FDS;
    ....
    if (err == NO_ERROR) {
         // 传输数据 【见流程2.6】
        err = writeTransactionData(BC_TRANSACTION, flags, handle, code, data, NULL);
    }

    if (err != NO_ERROR) {
        if (reply) reply->setError(err);
        return (mLastError = err);
    }

    if ((flags & TF_ONE_WAY) == 0) {
        // flags = 0 进入该分支
        if (reply) {
            // 等待响应 【见流程2.7】
            err = waitForResponse(reply);
        } else {
            Parcel fakeReply;
            err = waitForResponse(&fakeReply);
        }

    } else {
        // 不需要响应消息的 binder 则进入该分支
        err = waitForResponse(NULL, NULL);
    }
    return err;
}
```

----

### 2.6 IPC.writeTransactionData
- 其中 handle 的值用来标识目的端，注册服务过程的目的端为 service manager，此处 handle = 0 所对应的是 binder_context_mgr_node 对象，正是 service manager 所对应的 binder 实体对象
- binder_transaction_data 结构体是 binder 驱动通信的数据结构，该过程最终是把 Binder 请求码 BC_TRANSACTION 和binder_transaction_data 结构体写入到 mOut

```
status_t IPCThreadState::writeTransactionData(int32_t cmd, uint32_t binderFlags,
    int32_t handle, uint32_t code, const Parcel& data, status_t* statusBuffer)
{
    binder_transaction_data tr;
    tr.target.ptr = 0;
    tr.target.handle = handle;  // handle = 0
    tr.code = code;             // code = CHECK_SERVICE_TRANSACTION
    tr.flags = binderFlags;     // binderFlags = 0
    tr.cookie = 0;
    tr.sender_pid = 0;
    tr.sender_euid = 0;

    // data 为记录 Media 服务信息的 Parcel 对象
    const status_t err = data.errorCheck();
    if (err == NO_ERROR) {
        tr.data_size = data.ipcDataSize();       // mDataSize
        tr.data.ptr.buffer = data.ipcData();     // mData
        tr.offsets_size = data.ipcObjectsCount()*sizeof(binder_size_t); // mObjectsSize
        tr.data.ptr.offsets = data.ipcObjects(); //  mObjects
    } else if (statusBuffer) {
        ...
    } else {
        return (mLastError = err);
    }

    mOut.writeInt32(cmd);         // cmd = BC_TRANSACTION
    mOut.write(&tr, sizeof(tr));  // 写入 binder_transaction_data 数据
    return NO_ERROR;
}
```

---

### 2.7 IPC.waitForResponse
```
status_t IPCThreadState::waitForResponse(Parcel *reply, status_t *acquireResult)
{
    int32_t cmd;
    int32_t err;

    while (1) {
        if ((err=talkWithDriver()) < NO_ERROR) break; // 【见流程2.8】
        err = mIn.errorCheck();
        if (err < NO_ERROR) break;
        if (mIn.dataAvail() == 0) continue;

        cmd = mIn.readInt32();
        switch (cmd) {
            case BR_TRANSACTION_COMPLETE: ...
            case BR_DEAD_REPLY: ...
            case BR_FAILED_REPLY: ...
            case BR_ACQUIRE_RESULT: ...
            case BR_REPLY:
            {
              binder_transaction_data tr;
              err = mIn.read(&tr, sizeof(tr));
              if (reply) {
                  if ((tr.flags & TF_STATUS_CODE) == 0) {
                      reply->ipcSetDataReference(
                          reinterpret_cast<const uint8_t*>(tr.data.ptr.buffer),
                          tr.data_size,
                          reinterpret_cast<const binder_size_t*>(tr.data.ptr.offsets),
                          tr.offsets_size/sizeof(binder_size_t),
                          freeBuffer, this);
                  } else {
                      ...
                  }
              }
            }
            goto finish;

            default:
                err = executeCommand(cmd);
                if (err != NO_ERROR) goto finish;
                break;
        }
    }
    ...
    return err;
}
```

---

### 2.8 IPC.talkWithDriver
- binder_write_read 结构体用来与 Binder 设备交换数据的结构, 通过 ioctl 与 mDriverFD 通信，是真正与 Binder 驱动进行数据读写交互的过程
- 先向 service manager 进程发送查询服务的请求(BR_TRANSACTION)，见 Binder系列3—启动ServiceManager
- 当 service manager 进程收到该命令后，会执行 do_find_service() 查询服务所对应的 handle，然后再 binder_send_reply() 应答发起者，发送 BC_REPLY 协议，然后调用 binder_transaction()，再向服务请求者的 Todo 队列插入事务

```
status_t IPCThreadState::talkWithDriver(bool doReceive)
{
    ...
    binder_write_read bwr;
    const bool needRead = mIn.dataPosition() >= mIn.dataSize();
    const size_t outAvail = (!doReceive || needRead) ? mOut.dataSize() : 0;

    bwr.write_size = outAvail;
    bwr.write_buffer = (uintptr_t)mOut.data();

    if (doReceive && needRead) {
        // 接收数据缓冲区信息的填充。如果以后收到数据，就直接填在 mIn 中了。
        bwr.read_size = mIn.dataCapacity();
        bwr.read_buffer = (uintptr_t)mIn.data();
    } else {
        bwr.read_size = 0;
        bwr.read_buffer = 0;
    }
    // 当读缓冲和写缓冲都为空，则直接返回
    if ((bwr.write_size == 0) && (bwr.read_size == 0)) return NO_ERROR;

    bwr.write_consumed = 0;
    bwr.read_consumed = 0;
    status_t err;
    do {
        // 通过 ioctl 不停的读写操作，跟 Binder Driver 进行通信【2.8.1】
        if (ioctl(mProcess->mDriverFD, BINDER_WRITE_READ, &bwr) >= 0)
            err = NO_ERROR;
        ...
    } while (err == -EINTR); // 当被中断，则继续执行
    ...
    return err;
}
```

#### 2.8.1 binder_transaction
- 这个过程非常重要，分两种情况来说：
  - 当请求服务的进程与服务属于不同进程，则为请求服务所在进程创建 binder_ref 对象，指向服务进程中的 binder_node
  - 当请求服务的进程与服务属于同一进程，则不再创建新对象，只是引用计数加 1，并且修改 type 为 BINDER_TYPE_BINDER 或 BINDER_TYPE_WEAK_BINDER

```
static void binder_transaction(struct binder_proc *proc,
               struct binder_thread *thread,
               struct binder_transaction_data *tr, int reply) {
    // 根据各种判定，获取以下信息：
    struct binder_thread *target_thread； // 目标线程
    struct binder_proc *target_proc；     // 目标进程
    struct binder_node *target_node；     // 目标 binder 节点
    struct list_head *target_list；       // 目标 TODO 队列
    wait_queue_head_t *target_wait；      // 目标等待队列
    ...

    // 分配两个结构体内存
    struct binder_transaction *t = kzalloc(sizeof(*t), GFP_KERNEL);
    struct binder_work *tcomplete = kzalloc(sizeof(*tcomplete), GFP_KERNEL);
    // 从 target_proc 分配一块 buffer
    t->buffer = binder_alloc_buf(target_proc, tr->data_size,

    for (; offp < off_end; offp++) {
        switch (fp->type) {
        case BINDER_TYPE_BINDER: ...
        case BINDER_TYPE_WEAK_BINDER: ...

        case BINDER_TYPE_HANDLE:
        case BINDER_TYPE_WEAK_HANDLE: {
          struct binder_ref *ref = binder_get_ref(proc, fp->handle,
                fp->type == BINDER_TYPE_HANDLE);
          ...
          // 此时运行在 servicemanager 进程，故 ref->node 是指向服务所在进程的 binder 实体，
          // 而 target_proc 为请求服务所在的进程，此时并不相等。
          if (ref->node->proc == target_proc) {
            if (fp->type == BINDER_TYPE_HANDLE)
              fp->type = BINDER_TYPE_BINDER;
            else
              fp->type = BINDER_TYPE_WEAK_BINDER;
            fp->binder = ref->node->ptr;
            fp->cookie = ref->node->cookie;  // BBinder 服务的地址
            binder_inc_node(ref->node, fp->type == BINDER_TYPE_BINDER, 0, NULL);

          } else {
            struct binder_ref *new_ref;
            // 请求服务所在进程并非服务所在进程，则为请求服务所在进程创建 binder_ref
            new_ref = binder_get_ref_for_node(target_proc, ref->node);
            fp->binder = 0;
            fp->handle = new_ref->desc; //重新赋予handle值
            fp->cookie = 0;
            binder_inc_ref(new_ref, fp->type == BINDER_TYPE_HANDLE, NULL);
          }
        } break;

        case BINDER_TYPE_FD: ...
        }
    }
    // 分别 target_list 和当前线程 TODO 队列插入事务
    t->work.type = BINDER_WORK_TRANSACTION;
    list_add_tail(&t->work.entry, target_list);
    tcomplete->type = BINDER_WORK_TRANSACTION_COMPLETE;
    list_add_tail(&tcomplete->entry, &thread->todo);
    if (target_wait)
        wake_up_interruptible(target_wait);
    return;
}
```

#### 2.8.2 binder_thread_read
```
binder_thread_read（...）{
    ...
    // 当线程 todo 队列有数据则执行往下执行；当线程 todo 队列没有数据，则进入休眠等待状态
    ret = wait_event_freezable(thread->wait, binder_has_thread_work(thread));
    ...
    while (1) {
        uint32_t cmd;
        struct binder_transaction_data tr;
        struct binder_work *w;
        struct binder_transaction *t = NULL;
        // 先从线程 todo 队列获取事务数据
        if (!list_empty(&thread->todo)) {
            w = list_first_entry(&thread->todo, struct binder_work, entry);
        // 线程 todo 队列没有数据, 则从进程 todo 对获取事务数据
        } else if (!list_empty(&proc->todo) && wait_for_proc_work) {
            ...
        }
        switch (w->type) {
            case BINDER_WORK_TRANSACTION:
                // 获取 transaction 数据
                t = container_of(w, struct binder_transaction, work);
                break;

            case : ...  
        }

        // 只有 BINDER_WORK_TRANSACTION 命令才能继续往下执行
        if (!t) continue;

        if (t->buffer->target_node) {
            ...
        } else {
            tr.target.ptr = NULL;
            tr.cookie = NULL;
            cmd = BR_REPLY; // 设置命令为 BR_REPLY
        }
        tr.code = t->code;
        tr.flags = t->flags;
        tr.sender_euid = t->sender_euid;

        if (t->from) {
            struct task_struct *sender = t->from->proc->tsk;
            // 当非 oneway 的情况下,将调用者进程的 pid 保存到 sender_pid
            tr.sender_pid = task_tgid_nr_ns(sender, current->nsproxy->pid_ns);
        } else {
            ...
        }

        tr.data_size = t->buffer->data_size;
        tr.offsets_size = t->buffer->offsets_size;
        tr.data.ptr.buffer = (void *)t->buffer->data +
                    proc->user_buffer_offset;
        tr.data.ptr.offsets = tr.data.ptr.buffer +
                    ALIGN(t->buffer->data_size,
                        sizeof(void *));

        // 将 cmd 和数据写回用户空间
        put_user(cmd, (uint32_t __user *)ptr);
        ptr += sizeof(uint32_t);
        copy_to_user(ptr, &tr, sizeof(tr));
        ptr += sizeof(tr);

        list_del(&t->work.entry);
        t->buffer->allow_user_free = 1;
        if (cmd == BR_TRANSACTION && !(t->flags & TF_ONE_WAY)) {
            ...
        } else {
            t->buffer->transaction = NULL;
            kfree(t); // 通信完成则运行释放
        }
        break;
    }
done:
    *consumed = ptr - buffer;
    if (proc->requested_threads + proc->ready_threads == 0 &&
        proc->requested_threads_started < proc->max_threads &&
        (thread->looper & (BINDER_LOOPER_STATE_REGISTERED |
         BINDER_LOOPER_STATE_ENTERED))) {
        proc->requested_threads++;
        // 生成 BR_SPAWN_LOOPER命令，用于创建新的线程
        put_user(BR_SPAWN_LOOPER, (uint32_t __user *)buffer)；
    }
    return 0;
}
```

---

### 2.9 readStrongBinder
```
sp<IBinder> Parcel::readStrongBinder() const
{
    sp<IBinder> val;
    //【见小节2.9.1】
    unflatten_binder(ProcessState::self(), *this, &val);
    return val;
}
```

---

#### 2.9.1 unflatten_binder
```
status_t unflatten_binder(const sp<ProcessState>& proc,
    const Parcel& in, sp<IBinder>* out)
{
    const flat_binder_object* flat = in.readObject(false);
    if (flat) {
        switch (flat->type) {
            case BINDER_TYPE_BINDER:
                // 当请求服务的进程与服务属于同一进程
                *out = reinterpret_cast<IBinder*>(flat->cookie);
                return finish_unflatten_binder(NULL, *flat, in);
            case BINDER_TYPE_HANDLE:
                // 请求服务的进程与服务属于不同进程【见2.9.2】
                *out = proc->getStrongProxyForHandle(flat->handle);
                // 创建 BpBinder 对象
                return finish_unflatten_binder(
                    static_cast<BpBinder*>(out->get()), *flat, in);
        }
    }
    return BAD_TYPE;
}
```

---

#### 2.9.2 getStrongProxyForHandle
- readStrongBinder 的功能是 flat_binder_object 解析并创建 BpBinder 对象

```
sp<IBinder> ProcessState::getStrongProxyForHandle(int32_t handle)
{
    sp<IBinder> result;

    AutoMutex _l(mLock);
    // 查找 handle 对应的资源项[2.9.3]
    handle_entry* e = lookupHandleLocked(handle);

    if (e != NULL) {
        IBinder* b = e->binder;
        if (b == NULL || !e->refs->attemptIncWeak(this)) {
            ...
            // 当 handle 值所对应的 IBinder 不存在或弱引用无效时，则创建 BpBinder 对象
            b = new BpBinder(handle);
            e->binder = b;
            if (b) e->refs = b->getWeakRefs();
            result = b;
        } else {
            result.force_set(b);
            e->refs->decWeak(this);
        }
    }
    return result;
}
```

---

#### 2.9.3 lookupHandleLocked
- 根据 handle 值来查找对应的 handle_entry

```
ProcessState::handle_entry* ProcessState::lookupHandleLocked(int32_t handle)
{
    const size_t N=mHandleToObject.size();
    // 当 handle 大于 mHandleToObject 的长度时，进入该分支
    if (N <= (size_t)handle) {
        handle_entry e;
        e.binder = NULL;
        e.refs = NULL;
        // 从 mHandleToObject 的第 N 个位置开始，插入 (handle + 1 - N) 个 e 到队列中
        status_t err = mHandleToObject.insertAt(e, N, handle + 1 - N);
        if (err < NO_ERROR) return NULL;
    }
    return &mHandleToObject.editItemAt(handle);
}
```

## 3. 总结
- 请求服务(getService)过程，就是向 servicemanager 进程查询指定服务，当执行 binder_transaction() 时，会区分请求服务所属进程情况
  - 当请求服务的进程与服务属于不同进程，则为请求服务所在进程创建 binder_ref 对象，指向服务进程中的 binder_node
    - 最终 readStrongBinder()，返回的是 BpBinder 对象
  - 当请求服务的进程与服务属于同一进程，则不再创建新对象，只是引用计数加 1，并且修改 type 为 BINDER_TYPE_BINDER 或 BINDER_TYPE_WEAK_BINDER
    - 最终 readStrongBinder()，返回的是 BBinder 对象的真实子类
    
---